In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cristinabaldor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cristinabaldor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
#reading in json file of intents and responses

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

171 documents
35 classes ['about', 'animalstudies', 'complaint', 'dadjokes', 'disclosefoi', 'discloseopa', 'disclosetravel', 'disclosewhy', 'facilities', 'foreigninfluence', 'goodbye', 'greeting', 'help', 'leaderora', 'leaderraa', 'leadersbusiness', 'learners', 'listserv', 'loginproblems', 'name', 'outsidefunding', 'positivity', 'provostsawards', 'researchadmin', 'researchdevelopment', 'thanks', 'udiscloseconfidential', 'udisclosefaculty', 'udisclosefi1', 'udisclosefi2', 'udiscloseregister', 'udiscloseteam', 'viceprovost', 'vprsgeneral', 'writing']
195 unique lemmatized words ["'s", ',', 'a', 'about', 'access', 'accounting', 'activity', 'administration', 'all', 'am', 'an', 'and', 'animal', 'anyone', 'apply', 'are', 'audit', 'auditing', 'available', 'award', 'before', 'business', 'bye', 'call', 'can', 'charge', 'complaint', 'compliance', 'confidential', 'core', 'could', 'count', 'declaring', 'disclose', 'disclosed', 'disclosure', 'discouraged', 'do', 'doe', 'doing', 'done', 'editing', '

In [10]:


# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
35/35 [==============================] - 0s 1ms/step - loss: 3.5542 - accuracy: 0.0369 
Epoch 2/200
35/35 [==============================] - 0s 1ms/step - loss: 3.4695 - accuracy: 0.1421
Epoch 3/200
35/35 [==============================] - 0s 1ms/step - loss: 3.3793 - accuracy: 0.1773
Epoch 4/200
35/35 [==============================] - 0s 1ms/step - loss: 3.2562 - accuracy: 0.1581
Epoch 5/200
35/35 [==============================] - 0s 1ms/step - loss: 3.0274 - accuracy: 0.2985
Epoch 6/200
35/35 [==============================] - 0s 1ms/step - loss: 2.7540 - accuracy: 0.3767
Epoch 7/200
35/35 [==============================] - 0s 1ms/step - loss: 2.4918 - accuracy: 0.3756
Epoch 8/200
35/35 [==============================] - 0s 1ms/step - loss: 2.0204 - accuracy: 0.5780
Epoch 9/200
35/35 [==============================] - 0s 1ms/step - loss: 1.6373 - accuracy: 0.6226
Epoch 10/200
35/35 [==============================] - 0s 1ms/step - loss: 1.3642 - accuracy: 0.7876
Epoch 11

35/35 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 83/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 84/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 85/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 86/200
35/35 [==============================] - 0s 998us/step - loss: 0.0041 - accuracy: 1.0000
Epoch 87/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 88/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 89/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 90/200
35/35 [==============================] - 0s 985us/step - loss: 0.0041 - accuracy: 1.0000
Epoch 91/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 92/

35/35 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 164/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 165/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 166/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 167/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 168/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 169/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 170/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 171/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 172/200
35/35 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoc